In [2]:
import numpy as np
import pandas as pd

from pathlib import Path

# Graph Cleanup
This notebook puts data in a convenient format for creating the final graph.

In [4]:
DATAPATH = Path('../data/processed/')
PRPATH = DATAPATH / 'pagerank_rev_edge.parquet' #Can change this depending on which pagerank output you want
KYCPATH = DATAPATH / 'kyc.parquet'
EPATH = DATAPATH / 'emt.parquet'
WPATH = DATAPATH / 'wire.parquet'

In [25]:
# Load all nodes
node_df = pd.read_parquet(PRPATH)

# Join with kyc data and add country column
kyc_df = pd.read_parquet(KYCPATH)
kyc_df['country'] = 'CA'
node_df = pd.merge(node_df, kyc_df, how='left', on='cust_id')

# Get additional countries from wiretransfer data
wdf = pd.read_parquet(WPATH)
s1 = wdf[['cust_id_sender', 'country_sender']].copy().rename(columns={'cust_id_sender':'cust_id', 'country_sender':'country'})
s2 = wdf[['cust_id_receiver', 'country_receiver']].copy().rename(columns={'cust_id_receiver':'cust_id', 'country_receiver':'country'})
countries = pd.concat([s1,s2])

node_df = pd.merge(node_df, countries, how='left', on='cust_id')
node_df['country_x'] = node_df['country_x'].combine_first(node_df['country_y'])
node_df['country'] = node_df['country_x']
node_df = node_df.drop(columns=['country_x', 'country_y'])

#exporting 
node_df.to_parquet(DATAPATH /'nodes.parquet', index=False)

In [20]:
node_df

,cust_id,score,name,gender,occupation,age,tenure,occ_wealth,occ_animal,occ_int,label,country_x,country_y
0,CUST10000513,0.000478,DR.LOUIS-GUY LEPAGE,male,Barista,27.0,1.0,0.0,0.0,0.0,0.0,CA,NaN
1,CUST10001032,0.003689,DR.TSVETKOVA ANZHELIKA ARTEMOVNA,female,Civil Servant,52.0,17.0,0.0,0.0,0.0,0.0,CA,CA
2,CUST10001522,0.000090,LAURIE LEMAY,female,Author,37.0,5.0,0.0,0.0,0.0,0.0,CA,NaN
3,CUST10001820,0.000410,MARCEL BÉRUBÉ,male,Electrician,45.0,16.0,0.0,0.0,0.0,0.0,CA,NaN
4,CUST10002122,0.000029,ERICA GONZALEZ,female,Student,18.0,0.0,0.0,0.0,0.0,0.0,CA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
337880,EXTERNAL999910,0.000015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US
337881,EXTERNAL999944,0.000146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337882,EXTERNAL999949,0.000395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337883,EXTERNAL999981,0.000167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
node_df

,cust_id,score,name,gender,occupation,age,tenure,occ_wealth,occ_animal,occ_int,label,country
0,CUST10000513,0.000478,DR.LOUIS-GUY LEPAGE,male,Barista,27.0,1.0,0.0,0.0,0.0,0.0,CA
1,CUST10001032,0.003689,DR.TSVETKOVA ANZHELIKA ARTEMOVNA,female,Civil Servant,52.0,17.0,0.0,0.0,0.0,0.0,CA
2,CUST10001522,0.000090,LAURIE LEMAY,female,Author,37.0,5.0,0.0,0.0,0.0,0.0,CA
3,CUST10001820,0.000410,MARCEL BÉRUBÉ,male,Electrician,45.0,16.0,0.0,0.0,0.0,0.0,CA
4,CUST10002122,0.000029,ERICA GONZALEZ,female,Student,18.0,0.0,0.0,0.0,0.0,0.0,CA
...,...,...,...,...,...,...,...,...,...,...,...,...
337880,EXTERNAL999910,0.000015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US
337881,EXTERNAL999944,0.000146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337882,EXTERNAL999949,0.000395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337883,EXTERNAL999981,0.000167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
